In [2]:
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
from shapely.geometry import shape, LineString, mapping
from shapely.ops import unary_union, polygonize
import glob
import pandas as pd
import csv
from datetime import datetime
import time


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\3\ipykernel_39508\3616590515.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:

In [3]:
# Define engines
engines = {
    'fiona': {'engine': 'fiona'},
    'pyogrio': {'engine': 'pyogrio'},
    'pyogrio+arrow': {'engine': 'pyogrio', 'use_arrow': True}
          
}

def write_log(log_path, log_entry):
    # Check if the file exists to determine if headers need to be written
    file_exists = os.path.isfile(log_path)

    # Open the log file in append mode
    with open(log_path, 'a', newline='') as csvfile:
        log_writer = csv.writer(csvfile)

        # Write the header if the file is new
        if not file_exists:
            log_writer.writerow(["uc_code","uc_name","ctry_code","TotalUCAreaHa",
                                 "UC_UA_AreaHa","UC_UA_Pop","UC_UA_NCM_AreaHa","UC_UA_NCM_Pop",
                                 "Perc_coverage_Area","Perc_coverage_Pop","TotalGreen_urbc_ncm_Ha",
                                 "TotalGreenQuietAreaHa","TotalGQAPop","TotalGQAPop_p","PercGreenQuietArea",
                                 "PercGreenQuietPop","PercGreenQuietPop_p","weighted_median_Ha","TotalGreenAreaHa",
                                 "green_excludedHa"])

        # Write the log entry
        log_writer.writerow(log_entry)

def create_log_entry(val1, val2, val3, val4, val5, val6, val7, val8, val9, val10, val11, val12, val13, val14, val15, val16, val17, val18):
    return [val1, val2, val3, val4, val5, val6, val7, val8, val9, val10, val11, val12, val13, val14, val15, val16, val17, val18]



In [4]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
step3_f = os.path.join(indata_f, 'OutputData', 'step3_SA', 'Dissolved')
outdata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData'
ua_data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'
# Load urban center delineation
path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3B4Copy.shp') 
urban_centers = gpd.read_file(path)

# NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')
# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery', 
                     **engines['pyogrio+arrow'],columns=['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry'])

# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

# GQA
path2FinalGQA = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData\step2_GQA_Final'

# FINAL OUTPUT TABLE
csv_name = 'GQATotals_DGREGIO_v01.csv'
csv_path = os.path.join(outdata_f, 'OutTables', csv_name)

In [5]:
csv_name = 'GQATotals_DGREGIO_v02.csv'
csv_path = os.path.join(outdata_f, 'OutTables', csv_name)
SA_ls = ['GEOSTAT21_222',
 'GEOSTAT21_297',
 'GEOSTAT21_334',
 'GEOSTAT21_363',
 'GEOSTAT21_405',
 'GEOSTAT21_408',
 'GEOSTAT21_511',
 'GEOSTAT21_650', 
 'GEOSTAT21_901',
 'GEOSTAT21_925']

In [13]:
uc_code = 'GEOSTAT21_222'

In [15]:
codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.tolist()

['NL001L3_S_GRAVENHAGE_UA2018_v013', 'NL003L3_ROTTERDAM_UA2018_v013']

In [ ]:
ua_path = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[0].strip()

In [16]:
SA_ls

['GEOSTAT21_222',
 'GEOSTAT21_297',
 'GEOSTAT21_334',
 'GEOSTAT21_363',
 'GEOSTAT21_405',
 'GEOSTAT21_408',
 'GEOSTAT21_511',
 'GEOSTAT21_650',
 'GEOSTAT21_901',
 'GEOSTAT21_925']

In [6]:
uc_code= 'GEOSTAT21_511'
GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
GQA = gpd.read_file(GQA_path)
GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})


In [7]:
GQA

,GQA_FID,country,fua_name,fua_code,HDENS_2011,code_2018,class_2018,noisy,area_m2,area_ha,area_km2,geometry
0,0,CH,Winterthur,CH006L1,GEOSTAT11_511,14100,Green urban areas,0.0,2.834500e+03,0.28,0.00,"POLYGON ((4224168.882 2709067.514, 4224134.384..."
1,1,CH,Winterthur,CH006L1,GEOSTAT11_511,14100,Green urban areas,0.0,1.315997e+04,1.32,0.01,"POLYGON ((4224446.691 2709329.687, 4224446.922..."
2,2,CH,Winterthur,CH006L1,GEOSTAT11_511,14100,Green urban areas,0.0,4.091220e+03,0.41,0.00,"POLYGON ((4223765.021 2709154.538, 4223810.556..."
3,3,CH,Winterthur,CH006L1,GEOSTAT11_511,14100,Green urban areas,0.0,6.448563e+03,0.64,0.01,"POLYGON ((4224714.983 2709507.782, 4224700.094..."
4,4,CH,Winterthur,CH006L1,GEOSTAT11_511,14100,Green urban areas,0.0,2.418179e+03,0.24,0.00,"POLYGON ((4222886.462 2708365.955, 4222873.837..."
...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,CH,Winterthur,CH006L1,GEOSTAT11_511,31000,Forests,0.0,1.021349e+06,102.13,1.02,"POLYGON ((4224802.298 2712672.525, 4224803.960..."
75,75,CH,Winterthur,CH006L1,GEOSTAT11_511,14100,Green urban areas,0.0,3.481498e+04,3.48,0.03,"POLYGON ((4227267.934 2712826.448, 4227275.274..."
76,76,CH,Winterthur,CH006L1,GEOSTAT11_511,31000,Forests,0.0,1.214946e+03,0.12,0.00,"POLYGON ((4227808.393 2712881.626, 4227807.835..."
77,77,CH,Winterthur,CH006L1,GEOSTAT11_511,31000,Forests,0.0,2.143437e+02,0.02,0.00,"POLYGON ((4222000.000 2708265.844, 4222253.582..."


In [6]:
if os.path.exists(csv_path): 
    outtbl = pd.read_csv(csv_path)
    #cities_ls = outtbl.uc_code.to_list()
else:
    cities_ls = []

#SA_ls = sorted(glob.glob(os.path.join(step3_f, '*.shp')))
#SA_ls.reverse()

# Print the list of shapefiles
for sa_name in SA_ls[:1]:
    #uc_code = os.path.basename(sa_name)[:-8]
    uc_code = sa_name
    uc_name = codes.query(f'HDENS_CLST=="{uc_code}"').HDENS_NAME.values[0].strip().replace(" ", "").replace("/", "-")
    timenow = time.strftime("%H:%M:%S", time.localtime())
    print(f'{uc_name} {uc_code} - {timenow}')
    
    if uc_code not in cities_ls:

        # Load urban center
        urban_center = urban_centers.query(f'HDENS_CLST=="{uc_code}"')
        ctry_code = urban_centers.query(f'HDENS_CLST=="{uc_code}"').CNTR_CODE.values[0]
        TotalUCArea = urban_center.geometry.area.sum()
        TotalUCAreaHa = round(TotalUCArea/10000,2)  

        # Erase those areas of SA overlapping the GUAs
        # Load QGA
        GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
        GQA = gpd.read_file(GQA_path)
        GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})
        if 'area' in GQA.columns:
            GQA['GQA_m2'] = GQA['area']
        elif 'Area' in GQA.columns:
            GQA['GQA_m2'] = GQA['Area']
        else:            
            GQA['GQA_m2'] = GQA.geometry.area
            GQA.to_file(GQA_path, driver='ESRI Shapefile')

        # Remove the outer areas from urban centers      
        file_path = os.path.join(outdata_f,'SA_GQA_urbc', f'{uc_code}_SA_GQA_urbc.shp')
        if not os.path.exists(file_path):
            # Remove GQA delineation from the SA
            file_path_SAd = os.path.join(step3_f, f'{uc_code}_SAd.shp')
            SA_GQA_total = gpd.read_file(file_path_SAd)
            SA_GQA_total = SA_GQA_total.merge(GQA[['GQA_FID', 'GQA_m2']], left_on='FID_', right_on='GQA_FID', how='left')
            SA_GQA_urbc = gpd.overlay(SA_GQA_total, urban_center, how='intersection')
            SA_GQA_urbc.to_file(file_path, driver='ESRI Shapefile')     
        else:
            SA_GQA_urbc = gpd.read_file(file_path)

        #### Final computations including population
        ## Compute Total Green Areas
        # Read Urban Atlas data
        ua_ls = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.tolist()
        if len(ua_ls)==1:
            ua_path = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[0].strip()
            file_path = os.path.join(ua_data_f, f'{ua_path}\Data\{ua_path}.gpkg')
            # Read the GeoPackage file
            ua = gpd.read_file(file_path, layer= ua_path[:-5], 
                        columns= ['code_2018','Pop2018','geometry'], engine='pyogrio', 
                        use_arrow=True, bbox= tuple(urban_center.total_bounds))
        else:
            ua_path1 = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[0].strip()
            ua_path2 = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[1].strip()
            file_path1 = os.path.join(ua_data_f, f'{ua_path1}\Data\{ua_path1}.gpkg')
            file_path2 = os.path.join(ua_data_f, f'{ua_path2}\Data\{ua_path2}.gpkg')
            # Read the GeoPackage file
            ua1 = gpd.read_file(file_path1, layer= ua_path1[:-5], 
                        columns= ['code_2018','Pop2018','geometry'], 
                        engine='pyogrio', 
                        use_arrow=True, bbox= tuple(urban_center.total_bounds))
            ua2 = gpd.read_file(file_path2, layer= ua_path2[:-5], 
                        columns= ['code_2018','Pop2018','geometry'], 
                        engine='pyogrio', 
                        use_arrow=True, bbox= tuple(urban_center.total_bounds))
            print ("loaded ua in urban city")
            ua1 = gpd.clip(ua1, urban_center)
            ua2 = gpd.clip(ua2, urban_center)
            ua = pd.concat([ua1, ua2], ignore_index=True)          
        
        ua_urbc = gpd.overlay(ua, urban_center, how='intersection')
        #ua_urbc = gpd.clip(ua, urban_center)
        UC_UA_Area = ua_urbc.geometry.area.sum()
        UC_UA_AreaHa = round(UC_UA_Area/10000,2)
        UC_UA_Pop = int(ua_urbc.Pop2018.sum())
        
        # area of analysis
        ua_urbc_agl = gpd.overlay(ua_urbc, agls, how='intersection')
        UC_UA_NCM_AreaHa = round(ua_urbc_agl.geometry.area.sum()/10000,2)
        UC_UA_NCM_Pop = int(ua_urbc_agl.Pop2018.sum())

        # Perc coverage area
        # UC_NCM_UA_Area/Ref Area*100
        Perc_coverage_Area = round((UC_UA_NCM_AreaHa/TotalUCAreaHa)*100,2)
        
        # Perc coverage popultation
        # UC_NCM_UA_Pop/UC_UA_Pop*100
        if UC_UA_Pop>0 and UC_UA_NCM_Pop>0:
            Perc_coverage_Pop = round((UC_UA_NCM_Pop/UC_UA_Pop)*100,2)
        else:
            Perc_coverage_Pop = 0
  
        # All green areas (GUAs and forests) in the urban center (including data outside the ncm)
        TotalGreenArea = ua_urbc.query('code_2018 == "14100" or code_2018 == "31000"').geometry.area.sum()
        TotalGreenAreaHa = round(TotalGreenArea/10000,2)

        # green in the study area
        ua_urbc_agl_green = ua_urbc_agl.query('code_2018 == "14100" or code_2018 == "31000"')
        TotalGreen_urbc_ncm_Ha = round(ua_urbc_agl_green.geometry.area.sum()/10000,2)
        # save to file
        file_path = os.path.join(outdata_f, 'Green_inUCUA_Aglos', f'{uc_code}_greenUCUANCM.shp')
        if not os.path.exists(file_path):
            ua_urbc_agl_green.to_file(file_path, driver='ESRI Shapefile')

        # Excluded areas = TotalGreen in the urban center - TotalGreen in the uc & ncm 
        green_excludedHa = TotalGreenAreaHa - TotalGreen_urbc_ncm_Ha
        
        # Total Green Quiet Areas
        TotalGreenQuietAreaHa = round(GQA.area.sum()/10000,2)

        # Percentage of Green Quiet Areas
        ## Computed as:
        ## % of green quiet areas =  TotalGQA/(TotalGreenAreas-Green excluded) * 100
        ### Quiet_Area/UC_NCM_UA_Green_Area*100
        if TotalGreenQuietAreaHa>0 and TotalGreen_urbc_ncm_Ha>0:
            PercGreenQuietArea = TotalGreenQuietAreaHa/(TotalGreen_urbc_ncm_Ha)*100
        else:
            PercGreenQuietArea = 0

        # Computing Population
        # Total Population with access to GQA
        SA_mask = SA_GQA_urbc[['geometry', 'HDENS_NAME', 'CNTR_CODE', 'HDENS_CLST']].dissolve(by='HDENS_CLST')
        SA_ua = gpd.overlay(SA_mask,ua,how='intersection')  
        TotalGQAPop = SA_ua.Pop2018.sum()

        # Perc Population
        # TotalGQAPop/UC_NCM_UA_Pop*100
        if TotalGQAPop>0 and UC_UA_NCM_Pop>0:
            PercGreenQuietPop = TotalGQAPop/UC_UA_NCM_Pop*100
        else:
            PercGreenQuietPop = 0
        
        # Computing the weighted median
        ua['UA_area_m2'] = round(ua.geometry.area,2)
        SA_GQA_ua = gpd.overlay(SA_GQA_urbc,ua,how='intersection')  
        if not SA_GQA_ua.empty:  
            SA_GQA_ua = SA_GQA_ua[['GQA_FID', 'GQA_m2','code_2018', 'Pop2018','UA_area_m2', 'geometry']]
            SA_GQA_ua['final_area_m2'] = round(SA_GQA_ua.geometry.area,2)
            SA_GQA_ua['area_prop'] = SA_GQA_ua['final_area_m2']/SA_GQA_ua['UA_area_m2'] # capture proportion from original geometry
            SA_GQA_ua['Pop2018_poly'] = SA_GQA_ua['Pop2018']*SA_GQA_ua['area_prop']
            SA_GQA_ua['Pop2018_final'] = round(SA_GQA_ua.Pop2018_poly,0)
            # compute the weighted median for the GQ area
            column_to_compute_median = 'GQA_m2'
            weighted_median_m2 = np.average(SA_GQA_ua[column_to_compute_median], weights=SA_GQA_ua['Pop2018_poly'])
            weighted_median_Ha = round(weighted_median_m2/10000,2)
        else:
            weighted_median_Ha = 0

        # Write values to output table
        log_entry = create_log_entry(uc_code , uc_name , ctry_code , TotalUCAreaHa ,
                                UC_UA_AreaHa , UC_UA_Pop , UC_UA_NCM_AreaHa , UC_UA_NCM_Pop,
                                Perc_coverage_Area , Perc_coverage_Pop , TotalGreen_urbc_ncm_Ha ,
                                TotalGreenQuietAreaHa , TotalGQAPop , PercGreenQuietArea ,
                                PercGreenQuietPop , weighted_median_Ha , TotalGreenAreaHa ,
                                green_excludedHa)
        write_log(csv_path, log_entry)

    


Pijnacker-BerkelenRodenrijs GEOSTAT21_222 - 12:58:59


NameError: name 'cities_ls' is not defined

In [23]:
path2FinalGQA

'P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\OutputData\\step2_GQA_Final'

In [24]:
GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
GQA = gpd.read_file(GQA_path)
GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})

In [25]:
GQA

,GQA_FID,country,fua_name,fua_code,HDENS_2011,code_2018,class_2018,noisy,area_m2,area_ha,area_km2,geometry
0,0,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,13808.748613,1.38,0.01,"POLYGON ((3939295.845 3226123.002, 3939296.034..."
1,1,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,17111.656148,1.71,0.02,"POLYGON ((3939471.161 3225355.472, 3939466.413..."
2,2,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,17111.656148,1.71,0.02,"POLYGON ((3939737.032 3225958.008, 3939736.829..."
3,3,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,5942.944856,0.59,0.01,"POLYGON ((3939272.854 3225486.249, 3939288.183..."
4,4,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,3133.432255,0.31,0.00,"POLYGON ((3940095.279 3225797.089, 3940103.536..."
5,5,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,104.173241,0.01,0.00,"POLYGON ((3939937.390 3225937.622, 3939940.270..."
6,6,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,14198.673231,1.42,0.01,"POLYGON ((3939667.008 3225555.351, 3939656.961..."
7,7,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,14198.673231,1.42,0.01,"POLYGON ((3939481.546 3225848.949, 3939478.322..."
8,8,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,628.655708,0.06,0.00,"POLYGON ((3939416.054 3225740.996, 3939416.015..."
9,9,NL,s' Gravenhage,NL001L3,GEOSTAT11_222,14100,Green urban areas,0.0,2990.883846,0.30,0.00,"POLYGON ((3940107.812 3225933.907, 3940158.807..."


In [ ]:
# Print the list of shapefiles
for sa_name in SA_ls[:1]:
    #uc_code = os.path.basename(sa_name)[:-8]
    uc_code = sa_name
    uc_name = codes.query(f'HDENS_CLST=="{uc_code}"').HDENS_NAME.values[0].strip().replace(" ", "").replace("/", "-")
    timenow = time.strftime("%H:%M:%S", time.localtime())
    print(f'{uc_name} {uc_code} - {timenow}')
    
    # Load urban center
    urban_center = urban_centers.query(f'HDENS_CLST=="{uc_code}"')
    ctry_code = urban_centers.query(f'HDENS_CLST=="{uc_code}"').CNTR_CODE.values[0]
    TotalUCArea = urban_center.geometry.area.sum()
    TotalUCAreaHa = round(TotalUCArea/10000,2)  

    # Erase those areas of SA overlapping the GUAs
    # Load QGA
    GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
    GQA = gpd.read_file(GQA_path)
    GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})
    if 'area' in GQA.columns:
        GQA['GQA_m2'] = GQA['area']
    elif 'Area' in GQA.columns:
        GQA['GQA_m2'] = GQA['Area']
    else:            
        GQA['GQA_m2'] = GQA.geometry.area
        GQA.to_file(GQA_path, driver='ESRI Shapefile')

    # Remove the outer areas from urban centers      
    file_path = os.path.join(outdata_f,'SA_GQA_urbc', f'{uc_code}_SA_GQA_urbc.shp')
    if not os.path.exists(file_path):
        # Remove GQA delineation from the SA
        file_path_SAd = os.path.join(step3_f, f'{uc_code}_SAd.shp')
        SA_GQA_total = gpd.read_file(file_path_SAd)
        SA_GQA_total = SA_GQA_total.merge(GQA[['GQA_FID', 'GQA_m2']], left_on='FID_', right_on='GQA_FID', how='left')
        SA_GQA_urbc = gpd.overlay(SA_GQA_total, urban_center, how='intersection')
        SA_GQA_urbc.to_file(file_path, driver='ESRI Shapefile')     
    else:
        SA_GQA_urbc = gpd.read_file(file_path)

    #### Final computations including population
    ## Compute Total Green Areas
    # Read Urban Atlas data
    ua_ls = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.tolist()
    if len(ua_ls)==1:
        ua_path = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[0].strip()
        file_path = os.path.join(ua_data_f, f'{ua_path}\Data\{ua_path}.gpkg')
        # Read the GeoPackage file
        ua = gpd.read_file(file_path, layer= ua_path[:-5], 
                    columns= ['code_2018','Pop2018','geometry'], engine='pyogrio', 
                    use_arrow=True, bbox= tuple(urban_center.total_bounds))
    else:
        ua_path1 = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[0].strip()
        ua_path2 = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[1].strip()
        file_path1 = os.path.join(ua_data_f, f'{ua_path1}\Data\{ua_path1}.gpkg')
        file_path2 = os.path.join(ua_data_f, f'{ua_path2}\Data\{ua_path2}.gpkg')
        # Read the GeoPackage file
        ua1 = gpd.read_file(file_path1, layer= ua_path1[:-5], 
                    columns= ['code_2018','Pop2018','geometry'], 
                    engine='pyogrio', 
                    use_arrow=True, bbox= tuple(urban_center.total_bounds))
        ua2 = gpd.read_file(file_path2, layer= ua_path2[:-5], 
                    columns= ['code_2018','Pop2018','geometry'], 
                    engine='pyogrio', 
                    use_arrow=True, bbox= tuple(urban_center.total_bounds))
        print ("loaded ua in urban city")
        ua1 = gpd.clip(ua1, urban_center)
        ua2 = gpd.clip(ua2, urban_center)
        ua = pd.concat([ua1, ua2], ignore_index=True)          
    
    ua_urbc = gpd.overlay(ua, urban_center, how='intersection')
    #ua_urbc = gpd.clip(ua, urban_center)
    UC_UA_Area = ua_urbc.geometry.area.sum()
    UC_UA_AreaHa = round(UC_UA_Area/10000,2)
    UC_UA_Pop = int(ua_urbc.Pop2018.sum())
    
    # area of analysis
    ua_urbc_agl = gpd.overlay(ua_urbc, agls, how='intersection')
    UC_UA_NCM_AreaHa = round(ua_urbc_agl.geometry.area.sum()/10000,2)
    UC_UA_NCM_Pop = int(ua_urbc_agl.Pop2018.sum())

    # Perc coverage area
    # UC_NCM_UA_Area/Ref Area*100
    Perc_coverage_Area = round((UC_UA_NCM_AreaHa/TotalUCAreaHa)*100,2)
    
    # Perc coverage popultation
    # UC_NCM_UA_Pop/UC_UA_Pop*100
    if UC_UA_Pop>0 and UC_UA_NCM_Pop>0:
        Perc_coverage_Pop = round((UC_UA_NCM_Pop/UC_UA_Pop)*100,2)
    else:
        Perc_coverage_Pop = 0

    # All green areas (GUAs and forests) in the urban center (including data outside the ncm)
    TotalGreenArea = ua_urbc.query('code_2018 == "14100" or code_2018 == "31000"').geometry.area.sum()
    TotalGreenAreaHa = round(TotalGreenArea/10000,2)

    # green in the study area
    ua_urbc_agl_green = ua_urbc_agl.query('code_2018 == "14100" or code_2018 == "31000"')
    TotalGreen_urbc_ncm_Ha = round(ua_urbc_agl_green.geometry.area.sum()/10000,2)
    # save to file
    file_path = os.path.join(outdata_f, 'Green_inUCUA_Aglos', f'{uc_code}_greenUCUANCM.shp')
    if not os.path.exists(file_path):
        ua_urbc_agl_green.to_file(file_path, driver='ESRI Shapefile')

    # Excluded areas = TotalGreen in the urban center - TotalGreen in the uc & ncm 
    green_excludedHa = TotalGreenAreaHa - TotalGreen_urbc_ncm_Ha
    
    # Total Green Quiet Areas
    TotalGreenQuietAreaHa = round(GQA.area.sum()/10000,2)

    # Percentage of Green Quiet Areas
    ## Computed as:
    ## % of green quiet areas =  TotalGQA/(TotalGreenAreas-Green excluded) * 100
    ### Quiet_Area/UC_NCM_UA_Green_Area*100
    if TotalGreenQuietAreaHa>0 and TotalGreen_urbc_ncm_Ha>0:
        PercGreenQuietArea = TotalGreenQuietAreaHa/(TotalGreen_urbc_ncm_Ha)*100
    else:
        PercGreenQuietArea = 0

    # Computing Population
    # Total Population with access to GQA
    SA_mask = SA_GQA_urbc[['geometry', 'HDENS_NAME', 'CNTR_CODE', 'HDENS_CLST']].dissolve(by='HDENS_CLST')
    SA_ua = gpd.overlay(SA_mask,ua,how='intersection')  
    TotalGQAPop = SA_ua.Pop2018.sum()

    # Perc Population
    # TotalGQAPop/UC_NCM_UA_Pop*100
    if TotalGQAPop>0 and UC_UA_NCM_Pop>0:
        PercGreenQuietPop = TotalGQAPop/UC_UA_NCM_Pop*100
    else:
        PercGreenQuietPop = 0
    
    # Computing the weighted median
    ua['UA_area_m2'] = round(ua.geometry.area,2)
    SA_GQA_ua = gpd.overlay(SA_GQA_urbc,ua,how='intersection')  
    if not SA_GQA_ua.empty:  
        SA_GQA_ua = SA_GQA_ua[['GQA_FID', 'GQA_m2','code_2018', 'Pop2018','UA_area_m2', 'geometry']]
        SA_GQA_ua['final_area_m2'] = round(SA_GQA_ua.geometry.area,2)
        SA_GQA_ua['area_prop'] = SA_GQA_ua['final_area_m2']/SA_GQA_ua['UA_area_m2'] # capture proportion from original geometry
        SA_GQA_ua['Pop2018_poly'] = SA_GQA_ua['Pop2018']*SA_GQA_ua['area_prop']
        SA_GQA_ua['Pop2018_final'] = round(SA_GQA_ua.Pop2018_poly,0)
        # compute the weighted median for the GQ area
        column_to_compute_median = 'GQA_m2'
        weighted_median_m2 = np.average(SA_GQA_ua[column_to_compute_median], weights=SA_GQA_ua['Pop2018_poly'])
        weighted_median_Ha = round(weighted_median_m2/10000,2)
    else:
        weighted_median_Ha = 0

    # Write values to output table
    log_entry = create_log_entry(uc_code , uc_name , ctry_code , TotalUCAreaHa ,
                            UC_UA_AreaHa , UC_UA_Pop , UC_UA_NCM_AreaHa , UC_UA_NCM_Pop,
                            Perc_coverage_Area , Perc_coverage_Pop , TotalGreen_urbc_ncm_Ha ,
                            TotalGreenQuietAreaHa , TotalGQAPop , PercGreenQuietArea ,
                            PercGreenQuietPop , weighted_median_Ha , TotalGreenAreaHa ,
                            green_excludedHa)
    write_log(csv_path, log_entry)

    


In [7]:
# Script adapted to the last computation of urban centers with multiple urban atlas



#SA_ls = sorted(glob.glob(os.path.join(step3_f, '*.shp')))
#SA_ls.reverse()

# Print the list of shapefiles
for sa_name in SA_ls[:1]:
    #uc_code = os.path.basename(sa_name)[:-8]
    uc_code = sa_name
    uc_name = codes.query(f'HDENS_CLST=="{uc_code}"').HDENS_NAME.values[0].strip().replace(" ", "").replace("/", "-")
    timenow = time.strftime("%H:%M:%S", time.localtime())
    print(f'{uc_name} {uc_code} - {timenow}')
    

    # Load urban center
    urban_center = urban_centers.query(f'HDENS_CLST=="{uc_code}"')
    ctry_code = urban_centers.query(f'HDENS_CLST=="{uc_code}"').CNTR_CODE.values[0]
    TotalUCArea = urban_center.geometry.area.sum()
    TotalUCAreaHa = round(TotalUCArea/10000,2)  

    # Erase those areas of SA overlapping the GUAs
    # Load QGA
    GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
    GQA = gpd.read_file(GQA_path)
    GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})
    if not 'GQA_m2' in GQA.columns:
        if 'area' in GQA.columns:
            GQA['GQA_m2'] = GQA['area']
        elif 'Area' in GQA.columns:
            GQA['GQA_m2'] = GQA['Area']
        else:
            
            GQA['GQA_m2'] = GQA.geometry.area
        GQA.to_file(GQA_path, driver='ESRI Shapefile')

    # Remove the outer areas from urban centers      
    file_path = os.path.join(outdata_f,'SA_GQA_urbc', f'{uc_code}_SA_GQA_urbc.shp')
    if not os.path.exists(file_path):
        # Remove GQA delineation from the SA
        file_path_SAd = os.path.join(step3_f, f'{uc_code}_SAd.shp')
        SA_GQA_total = gpd.read_file(file_path_SAd)
        #SA_GQA_total = SA_GQA_total.merge(GQA[['GQA_FID', 'GQA_m2']], left_on='FID_', right_on='GQA_FID', how='left')
        SA_GQA_total = SA_GQA_total.merge(GQA[['GQA_FID', 'GQA_m2']], left_on='FID_1', right_on='GQA_FID', how='left')
        SA_GQA_urbc = gpd.overlay(SA_GQA_total, urban_center, how='intersection')
        SA_GQA_urbc.to_file(file_path, driver='ESRI Shapefile')     
    else:
        SA_GQA_urbc = gpd.read_file(file_path)

    #### Final computations including population
    ## Compute Total Green Areas
    # Read Urban Atlas data
    ua_path = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[0].strip()
    file_path = os.path.join(ua_data_f, f'{ua_path}\Data\{ua_path}.gpkg')
    # Read the GeoPackage file
    ua = gpd.read_file(file_path, layer= ua_path[:-5], 
                columns= ['code_2018','Pop2018','geometry'], engine='pyogrio', 
                use_arrow=True, bbox= tuple(urban_center.total_bounds))
    
    ua_urbc = gpd.overlay(ua, urban_center, how='intersection')
    #ua_urbc = gpd.clip(ua, urban_center)
    UC_UA_Area = ua_urbc.geometry.area.sum()
    UC_UA_AreaHa = round(UC_UA_Area/10000,2)
    UC_UA_Pop = int(ua_urbc.Pop2018.sum())
    
    # area of analysis
    ua_urbc_agl = gpd.overlay(ua_urbc, agls, how='intersection')
    UC_UA_NCM_AreaHa = round(ua_urbc_agl.geometry.area.sum()/10000,2)
    UC_UA_NCM_Pop = int(ua_urbc_agl.Pop2018.sum())

    # Perc coverage area
    # UC_NCM_UA_Area/Ref Area*100
    Perc_coverage_Area = round((UC_UA_NCM_AreaHa/TotalUCAreaHa)*100,2)
    
    # Perc coverage popultation
    # UC_NCM_UA_Pop/UC_UA_Pop*100
    if UC_UA_Pop>0 and UC_UA_NCM_Pop>0:
        Perc_coverage_Pop = round((UC_UA_NCM_Pop/UC_UA_Pop)*100,2)
    else:
        Perc_coverage_Pop = 0

    # All green areas (GUAs and forests) in the urban center (including data outside the ncm)
    TotalGreenArea = ua_urbc.query('code_2018 == "14100" or code_2018 == "31000"').geometry.area.sum()
    TotalGreenAreaHa = round(TotalGreenArea/10000,2)

    # green in the study area
    ua_urbc_agl_green = ua_urbc_agl.query('code_2018 == "14100" or code_2018 == "31000"')
    TotalGreen_urbc_ncm_Ha = round(ua_urbc_agl_green.geometry.area.sum()/10000,2)
    # save to file
    file_path = os.path.join(outdata_f, 'Green_inUCUA_Aglos', f'{uc_code}_greenUCUANCM.shp')
    if not os.path.exists(file_path):
        ua_urbc_agl_green.to_file(file_path, driver='ESRI Shapefile')

    # Excluded areas = TotalGreen in the urban center - TotalGreen in the uc & ncm 
    green_excludedHa = TotalGreenAreaHa - TotalGreen_urbc_ncm_Ha
    
    # Total Green Quiet Areas
    TotalGreenQuietAreaHa = round(GQA.area.sum()/10000,2)

    # Percentage of Green Quiet Areas
    ## Computed as:
    ## % of green quiet areas =  TotalGQA/(TotalGreenAreas-Green excluded) * 100
    ### Quiet_Area/UC_NCM_UA_Green_Area*100
    if TotalGreenQuietAreaHa>0 and TotalGreen_urbc_ncm_Ha>0:
        PercGreenQuietArea = TotalGreenQuietAreaHa/(TotalGreen_urbc_ncm_Ha)*100
    else:
        PercGreenQuietArea = 0

    # Computing Population
    # Total Population with access to GQA
    SA_mask = SA_GQA_urbc[['geometry', 'HDENS_NAME', 'CNTR_CODE', 'HDENS_CLST']].dissolve(by='HDENS_CLST')
    SA_ua = gpd.overlay(SA_mask,ua,how='intersection')  
    TotalGQAPop = SA_ua.Pop2018.sum()

    # Perc Population
    # TotalGQAPop/UC_NCM_UA_Pop*100
    if TotalGQAPop>0 and UC_UA_NCM_Pop>0:
        PercGreenQuietPop = TotalGQAPop/UC_UA_NCM_Pop*100
    else:
        PercGreenQuietPop = 0
    
    # Computing the weighted median
    ua['UA_area_m2'] = round(ua.geometry.area,2)
    SA_GQA_ua = gpd.overlay(SA_GQA_urbc,ua,how='intersection')  
    if not SA_GQA_ua.empty:  
        SA_GQA_ua = SA_GQA_ua[['GQA_FID', 'GQA_m2','code_2018', 'Pop2018','UA_area_m2', 'geometry']]
        SA_GQA_ua['final_area_m2'] = round(SA_GQA_ua.geometry.area,2)
        SA_GQA_ua['area_prop'] = SA_GQA_ua['final_area_m2']/SA_GQA_ua['UA_area_m2'] # capture proportion from original geometry
        SA_GQA_ua['Pop2018_poly'] = SA_GQA_ua['Pop2018']*SA_GQA_ua['area_prop']
        SA_GQA_ua['Pop2018_final'] = round(SA_GQA_ua.Pop2018_poly,0)
        # compute the weighted median for the GQ area
        column_to_compute_median = 'GQA_m2'
        weighted_median_m2 = np.average(SA_GQA_ua[column_to_compute_median], weights=SA_GQA_ua['Pop2018_poly'])
        weighted_median_Ha = round(weighted_median_m2/10000,2)
    else:
        weighted_median_Ha = 0

    # Write values to output table
    log_entry = create_log_entry(uc_code , uc_name , ctry_code , TotalUCAreaHa ,
                            UC_UA_AreaHa , UC_UA_Pop , UC_UA_NCM_AreaHa , UC_UA_NCM_Pop,
                            Perc_coverage_Area , Perc_coverage_Pop , TotalGreen_urbc_ncm_Ha ,
                            TotalGreenQuietAreaHa , TotalGQAPop , PercGreenQuietArea ,
                            PercGreenQuietPop , weighted_median_Ha , TotalGreenAreaHa ,
                            green_excludedHa)
    write_log(csv_path, log_entry)

    


Pijnacker-BerkelenRodenrijs GEOSTAT21_222 - 12:59:42


PermissionError: [Errno 13] Permission denied: 'P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\OutputData\\OutTables\\GQATotals_DGREGIO_v02.csv'

In [8]:
write_log(csv_path, log_entry)

In [9]:
GQA.head()

,GQA_FID,FID_1,FacilityID,Name,FromBreak,ToBreak,Shape_Leng,Shape_Area,geometry,GQA_m2
0,0,0,7,Location 7 : 0 - 400,0.0,400.0,4291.586109,137725.0,"POLYGON ((3939402.287 3226368.752, 3939405.923...",264511.877586
1,1,1,10,Location 10 : 0 - 400,0.0,400.0,6503.197989,218637.5,"POLYGON ((3939442.287 3225603.752, 3939421.870...",391339.171264
2,2,2,32,Location 32 : 0 - 400,0.0,400.0,5556.720822,190187.5,"POLYGON ((3939505.620 3226175.418, 3939522.287...",247356.878735
3,3,3,39,Location 39 : 0 - 400,0.0,400.0,4675.505455,87625.0,"POLYGON ((3939266.892 3225516.383, 3939267.287...",263556.039692
4,4,4,53,Location 53 : 0 - 400,0.0,400.0,4412.929354,145137.5,"POLYGON ((3940221.404 3225969.340, 3940281.012...",216491.174753


In [10]:
SA_GQA_total

,FID_1,geometry
0,0,"POLYGON ((4224321.026 2709454.176, 4224331.026..."
1,1,"POLYGON ((4224481.026 2709589.176, 4224491.026..."
2,2,"POLYGON ((4223741.026 2709459.176, 4223746.026..."
3,3,"POLYGON ((4224661.026 2709914.176, 4224671.026..."
4,4,"POLYGON ((4222946.026 2708809.176, 4222966.026..."
...,...,...
74,74,"POLYGON ((4224666.026 2712954.176, 4224686.026..."
75,75,"POLYGON ((4226981.026 2713114.176, 4226996.026..."
76,76,"POLYGON ((4227931.026 2713314.176, 4227951.026..."
77,77,"POLYGON ((4222241.026 2708069.176, 4222421.026..."


In [10]:
codes.query(f'HDENS_CLST=="{uc_code}"').HDENS_NAME.values[0].strip().replace(" ", "").replace("/", "-")

'Pijnacker-BerkelenRodenrijs'

In [12]:
ua_urbc_agl_green

NameError: name 'ua_urbc_agl_green' is not defined

In [ ]:
file_path = os.path.join(outdata_f, 'Green_inUCUA_Aglos', f'{uc_code}_greenUCUANCM.shp')
if not os.path.exists(file_path):
    ua_urbc_agl_green.to_file(file_path, driver='ESRI Shapefile')

In [6]:
sa_name

'GEOSTAT21_222'

In [4]:
SA_ls = sorted(glob.glob(os.path.join(step3_f, '*.shp')))

In [27]:
SA_ls[159:160]

['P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\OutputData\\step3_SA\\Dissolved\\GEOSTAT21_441_SAd.shp']

In [33]:
if os.path.exists(csv_path): 
    outtbl = pd.read_csv(csv_path)
    cities_ls = outtbl.uc_code.to_list()
else:
    cities_ls = []

SA_ls = sorted(glob.glob(os.path.join(step3_f, '*.shp')))
#SA_ls.reverse()

# Print the list of shapefiles
for sa_name in SA_ls:
    uc_code = os.path.basename(sa_name)[:-8]
    uc_name = codes.query(f'HDENS_CLST=="{uc_code}"').HDENS_NAME.values[0]
    timenow = time.strftime("%H:%M:%S", time.localtime())
    print(f'{uc_name} {uc_code} - {timenow}')
    
    if uc_code not in cities_ls:

        # Load urban center
        urban_center = urban_centers.query(f'HDENS_CLST=="{uc_code}"')
        ctry_code = urban_centers.query(f'HDENS_CLST=="{uc_code}"').CNTR_CODE.values[0]
        TotalUCArea = urban_center.geometry.area.sum()
        TotalUCAreaHa = round(TotalUCArea/10000,2)  

        # Erase those areas of SA overlapping the GUAs
        # Load QGA
        path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
        GQA = gpd.read_file(path)
        GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})
        if 'area' in GQA.columns:
            GQA['GQA_m2'] = GQA['area']
        else:
            GQA['GQA_m2'] = GQA['Area']

        # Remove the outer areas from urban centers      
        file_path = os.path.join(outdata_f,'SA_GQA_urbc', f'{uc_code}_SA_GQA_urbc.shp')
        if not os.path.exists(file_path):
            # Remove GQA delineation from the SA
            file_path_SAd = os.path.join(step3_f, f'{uc_code}_SAd.shp')
            SA_GQA_total = gpd.read_file(file_path_SAd)
            SA_GQA_total = SA_GQA_total.merge(GQA[['GQA_FID', 'GQA_m2']], left_on='FID_', right_on='GQA_FID', how='left')
            SA_GQA_urbc = gpd.overlay(SA_GQA_total, urban_center, how='intersection')
            SA_GQA_urbc.to_file(file_path, driver='ESRI Shapefile')     
        else:
            SA_GQA_urbc = gpd.read_file(file_path)

        #### Final computations including population
        ## Compute Total Green Areas
        # Read Urban Atlas data
        ua_path = codes.query(f'HDENS_CLST=="{uc_code}"').UA2018.values[0].strip()
        file_path = os.path.join(ua_data_f, f'{ua_path}\Data\{ua_path}.gpkg')
        # Read the GeoPackage file
        ua = gpd.read_file(file_path, layer= ua_path[:-5], 
                    columns= ['code_2018','Pop2018','geometry'], engine='pyogrio', 
                    use_arrow=True, bbox= tuple(urban_center.total_bounds))
        
        ua_urbc = gpd.overlay(ua, urban_center, how='intersection')
        #ua_urbc = gpd.clip(ua, urban_center)
        UC_UA_Area = ua_urbc.geometry.area.sum()
        UC_UA_AreaHa = round(UC_UA_Area/10000,2)
        UC_UA_Pop = int(ua_urbc.Pop2018.sum())
        
        # area of analysis
        ua_urbc_agl = gpd.overlay(ua_urbc, agls, how='intersection')
        UC_UA_NCM_AreaHa = round(ua_urbc_agl.geometry.area.sum()/10000,2)
        UC_UA_NCM_Pop = int(ua_urbc_agl.Pop2018.sum())

        # Perc coverage area
        # UC_NCM_UA_Area/Ref Area*100
        Perc_coverage_Area = round((UC_UA_NCM_AreaHa/TotalUCAreaHa)*100,2)
        
        # Perc coverage popultation
        # UC_NCM_UA_Pop/UC_UA_Pop*100
        if UC_UA_Pop>0 and UC_UA_NCM_Pop>0:
            Perc_coverage_Pop = round((UC_UA_NCM_Pop/UC_UA_Pop)*100,2)
        else:
            Perc_coverage_Pop = 0
  
        # All green areas (GUAs and forests) in the urban center (including data outside the ncm)
        TotalGreenArea = ua_urbc.query('code_2018 == "14100" or code_2018 == "31000"').geometry.area.sum()
        TotalGreenAreaHa = round(TotalGreenArea/10000,2)

        # green in the study area
        ua_urbc_agl_green = ua_urbc_agl.query('code_2018 == "14100" or code_2018 == "31000"')
        TotalGreen_urbc_ncm_Ha = round(ua_urbc_agl_green.geometry.area.sum()/10000,2)
        # save to file
        file_path = os.path.join(outdata_f, 'Green_inUCUA_Aglos', f'{uc_code}_greenUCUANCM.shp')
        if not os.path.exists(file_path):
            ua_urbc_agl_green.to_file(file_path, driver='ESRI Shapefile')

        # Excluded areas = TotalGreen in the urban center - TotalGreen in the uc & ncm 
        green_excludedHa = TotalGreenAreaHa - TotalGreen_urbc_ncm_Ha
        
        # Total Green Quiet Areas
        TotalGreenQuietAreaHa = round(GQA.area.sum()/10000,2)

        # Percentage of Green Quiet Areas
        ## Computed as:
        ## % of green quiet areas =  TotalGQA/(TotalGreenAreas-Green excluded) * 100
        ### Quiet_Area/UC_NCM_UA_Green_Area*100
        if TotalGreenQuietAreaHa>0 and TotalGreen_urbc_ncm_Ha>0:
            PercGreenQuietArea = TotalGreenQuietAreaHa/(TotalGreen_urbc_ncm_Ha)*100
        else:
            PercGreenQuietArea = 0

        # Computing Population
        # Total Population with access to GQA
        SA_mask = SA_GQA_urbc[['geometry', 'HDENS_NAME', 'CNTR_CODE', 'HDENS_CLST']].dissolve(by='HDENS_CLST')
        ua['UA_area_m2'] = round(ua.geometry.area,2)
        SA_ua = gpd.overlay(SA_mask,ua,how='intersection')  
        SA_ua['final_area_m2'] = round(SA_ua.geometry.area,2)
        SA_ua['area_prop'] = SA_ua['final_area_m2']/SA_ua['UA_area_m2']
        SA_ua['Pop2018_poly'] = SA_ua['Pop2018']*SA_ua['area_prop']
        TotalGQAPop = round(SA_ua.Pop2018.sum(),0)
        TotalGQAPop_p = round(SA_ua.Pop2018_poly.sum(),0)
        
        # Computing the weighted median        
        SA_GQA_ua = gpd.overlay(SA_GQA_urbc,ua,how='intersection')  
        if not SA_GQA_ua.empty:  
            SA_GQA_ua = SA_GQA_ua[['GQA_FID', 'GQA_m2','code_2018', 'Pop2018','UA_area_m2', 'geometry']]
            SA_GQA_ua['final_area_m2'] = round(SA_GQA_ua.geometry.area,2)
            SA_GQA_ua['area_prop'] = SA_GQA_ua['final_area_m2']/SA_GQA_ua['UA_area_m2'] # capture proportion from original geometry
            SA_GQA_ua['Pop2018_poly'] = SA_GQA_ua['Pop2018']*SA_GQA_ua['area_prop']

            # compute the weighted median for the GQ area
            column_to_compute_median = 'GQA_m2'
            weighted_median_m2 = np.average(SA_GQA_ua[column_to_compute_median], weights=SA_GQA_ua['Pop2018_poly'])
            weighted_median_Ha = round(weighted_median_m2/10000,2)
        else:
            weighted_median_Ha = 0

        # Perc Population
        # TotalGQAPop/UC_NCM_UA_Pop*100
        if TotalGQAPop>0 and UC_UA_NCM_Pop>0:
            PercGreenQuietPop = TotalGQAPop/UC_UA_NCM_Pop*100
        else:
            PercGreenQuietPop = 0
        
        if TotalGQAPop_p>0 and UC_UA_NCM_Pop>0:
            PercGreenQuietPop_p = TotalGQAPop_p/UC_UA_NCM_Pop*100
        else:
            PercGreenQuietPop_p = 0

        # Write values to output table
        log_entry = create_log_entry(uc_code , uc_name , ctry_code , TotalUCAreaHa ,
                                UC_UA_AreaHa , UC_UA_Pop , UC_UA_NCM_AreaHa , UC_UA_NCM_Pop,
                                Perc_coverage_Area , Perc_coverage_Pop , TotalGreen_urbc_ncm_Ha ,
                                TotalGreenQuietAreaHa , TotalGQAPop, TotalGQAPop_p, PercGreenQuietArea ,
                                PercGreenQuietPop ,PercGreenQuietPop_p, weighted_median_Ha , TotalGreenAreaHa ,
                                green_excludedHa)
        write_log(csv_path, log_entry)

    


Oulu / Uleåborg GEOSTAT21_002 - 15:28:56
Umeå GEOSTAT21_003 - 15:29:27


Q:\UserTemp\ubach\AppData\Local\Temp\40\ipykernel_126376\2347343578.py:57: UserWarning: `keep_geom_type=True` in overlay resulted in 2 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ua_urbc = gpd.overlay(ua, urban_center, how='intersection')


Trondheim GEOSTAT21_004 - 15:30:36
Jyväskylä GEOSTAT21_005 - 15:31:24
Tampere / Tammerfors GEOSTAT21_006 - 15:31:52
Turku / Åbo GEOSTAT21_007 - 15:34:17
Bergen GEOSTAT21_011 - 15:36:11
Tallinn GEOSTAT21_012 - 15:36:50
Uppsala GEOSTAT21_013 - 15:39:06


Q:\UserTemp\ubach\AppData\Local\Temp\40\ipykernel_126376\2347343578.py:57: UserWarning: `keep_geom_type=True` in overlay resulted in 2 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ua_urbc = gpd.overlay(ua, urban_center, how='intersection')


Västerås GEOSTAT21_016 - 15:41:26


...

Logroño GEOSTAT21_694 - 22:54:51
Porto GEOSTAT21_706 - 22:55:03
Valladolid GEOSTAT21_712 - 22:58:04
Torrejón de Ardoz GEOSTAT21_768 - 22:58:58
Madrid GEOSTAT21_773 - 23:05:20
Fuenlabrada GEOSTAT21_774 - 23:24:06
Castellón de la Plana/Castelló de la Plana GEOSTAT21_800 - 23:30:25
Valencia GEOSTAT21_809 - 23:30:34
Lefkosia GEOSTAT21_843 - 23:35:15
Lemesos GEOSTAT21_856 - 23:35:49
Valletta GEOSTAT21_867 - 23:36:15


Q:\UserTemp\ubach\AppData\Local\Temp\40\ipykernel_126376\2347343578.py:57: UserWarning: `keep_geom_type=True` in overlay resulted in 26 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ua_urbc = gpd.overlay(ua, urban_center, how='intersection')


Helsinki GEOSTAT21_901 - 23:37:34
Oslo GEOSTAT21_902 - 23:41:37
København GEOSTAT21_903 - 23:46:12
Dublin GEOSTAT21_904 - 23:52:59
Utrecht GEOSTAT21_905 - 00:07:49
Paris GEOSTAT21_906 - 00:09:07


Q:\UserTemp\ubach\AppData\Local\Temp\40\ipykernel_126376\2347343578.py:57: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ua_urbc = gpd.overlay(ua, urban_center, how='intersection')


Augsburg GEOSTAT21_907 - 02:13:47
Lahti GEOSTAT21_911 - 02:15:10
Jönköping / Huskvarna GEOSTAT21_914 - 02:15:30
Neu Wulmstorf / Neugraben-Fischbek / Hausbruch GEOSTAT21_919 - 02:16:27
Scharwoude / Heerhugowaard GEOSTAT21_922 - 02:19:04
Wuppertal GEOSTAT21_925 - 02:19:26
Saint-Priest / Mions GEOSTAT21_945 - 02:19:31


In [ ]:
write_log(csv_path, log_entry)

In [34]:
tbl = pd.read_csv(csv_path) 
tbl.head()

,uc_code,uc_name,ctry_code,TotalUCAreaHa,UC_UA_AreaHa,UC_UA_Pop,UC_UA_NCM_AreaHa,UC_UA_NCM_Pop,Perc_coverage_Area,Perc_coverage_Pop,TotalGreen_urbc_ncm_Ha,TotalGreenQuietAreaHa,TotalGQAPop,PercGreenQuietArea,PercGreenQuietPop,weighted_median_Ha,TotalGreenAreaHa,green_excludedHa
0,GEOSTAT21_002,Oulu / Uleåborg,FI,2900.0,2900.00,76701,2900.00,76701,100.00,100.00,715.95,340.85,38825.0,47.608073,50.618636,2.37,715.95,0.00
1,GEOSTAT21_003,Umeå,SE,3600.0,3600.00,85430,3600.00,85430,100.00,100.00,949.30,421.74,27183.0,44.426419,31.819033,7.67,949.30,0.00
2,GEOSTAT21_004,Trondheim,NO,7000.0,6792.94,177126,6669.17,177123,95.27,100.00,1289.85,954.85,79888.0,74.027988,45.103120,6.68,1307.75,17.90
3,GEOSTAT21_005,Jyväskylä,FI,2700.0,2700.00,64379,2700.00,64379,100.00,100.00,603.42,369.46,29175.0,61.227669,45.317573,5.46,603.42,0.00
4,GEOSTAT21_006,Tampere / Tammerfors,FI,8200.0,8200.00,196152,8159.01,195838,99.50,99.84,1796.15,1016.24,89868.0,56.578794,45.888949,3.37,1801.40,5.25


In [35]:
tbl.to_csv(csv_path, index=False, encoding="utf-8")

In [31]:
log_entry

['GEOSTAT21_441',
 'Heilbronn',
 'DE',
 2400.0,
 2400.0,
 100177,
 2122.63,
 93871,
 88.44,
 93.71,
 114.79,
 88.28,
 22269.0,
 76.90565380259604,
 23.722981538494317,
 4.14,
 122.19,
 7.3999999999999915]

In [8]:
SA_GQA_ua

,GQA_FID,GQA_m2,code_2018,Pop2018,UA_area_m2,geometry,final_area_m2,area_prop,Pop2018_poly,Pop2018_final
0,0,91687.918884,12220,0,3.443834e+08,"POLYGON ((3737532.895 2880000.000, 3737543.421...",11847.85,0.000034,0.000000,0.0
1,1,1208.064559,12220,0,3.443834e+08,"POLYGON ((3737342.851 2880250.228, 3737332.879...",7242.83,0.000021,0.000000,0.0
2,2,129614.521449,12220,0,3.443834e+08,"MULTIPOLYGON (((3737949.361 2880767.997, 37379...",21985.74,0.000064,0.000000,0.0
3,3,10782.694370,12220,0,3.443834e+08,"POLYGON ((3737683.579 2880757.022, 3737681.225...",37175.81,0.000108,0.000000,0.0
4,4,4240.194748,12220,0,3.443834e+08,"MULTIPOLYGON (((3737402.585 2880764.695, 37373...",26280.71,0.000076,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
1874,89,28513.164237,11210,54,7.287360e+03,"POLYGON ((3731000.000 2885630.552, 3731001.351...",7231.95,0.992396,53.589407,54.0
1875,89,28513.164237,11210,64,6.734230e+03,"POLYGON ((3731126.802 2885621.155, 3731121.609...",6734.23,1.000000,64.000000,64.0
1876,89,28513.164237,11210,51,8.317960e+03,"POLYGON ((3731181.439 2885511.912, 3731168.165...",8317.96,1.000000,51.000000,51.0
1877,89,28513.164237,11210,80,1.073271e+04,"POLYGON ((3731239.168 2885485.170, 3731222.682...",10732.71,1.000000,80.000000,80.0


In [9]:
SA_GQA_ua.Pop2018_poly.sum()

81714.87435043047

In [10]:
file_path_SA_GQA_ua = os.path.join(outdata_f,'SA_GQA_ua', f'{uc_code}_SA_GQA_ua.shp')
SA_GQA_ua.to_file(file_path_SA_GQA_ua, driver='ESRI Shapefile')   

Q:\UserTemp\ubach\AppData\Local\Temp\40\ipykernel_126376\3988194404.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  SA_GQA_ua.to_file(file_path_SA_GQA_ua, driver='ESRI Shapefile')


In [12]:
file_path_SA_GQA_ua = os.path.join(outdata_f,'SA', f'{uc_code}_SA_mask.shp')
SA_mask.to_file(file_path_SA_GQA_ua, driver='ESRI Shapefile')   

In [13]:
SA_ua

,HDENS_NAME,CNTR_CODE,code_2018,Pop2018,geometry
0,Maurepas / Montigny-le-Bretonneux,FR,21000,0,"POLYGON ((3730027.052 2884037.266, 3730026.813..."
1,Maurepas / Montigny-le-Bretonneux,FR,12220,0,"MULTIPOLYGON (((3739213.184 2881792.216, 37392..."
2,Maurepas / Montigny-le-Bretonneux,FR,11210,389,"POLYGON ((3729103.938 2882435.228, 3729076.019..."
3,Maurepas / Montigny-le-Bretonneux,FR,11210,160,"MULTIPOLYGON (((3728958.938 2882510.228, 37289..."
4,Maurepas / Montigny-le-Bretonneux,FR,12100,5,"POLYGON ((3729053.938 2882435.228, 3729063.938..."
...,...,...,...,...,...
872,Maurepas / Montigny-le-Bretonneux,FR,11210,54,"POLYGON ((3731000.000 2885630.552, 3731001.351..."
873,Maurepas / Montigny-le-Bretonneux,FR,11210,64,"POLYGON ((3731126.802 2885621.155, 3731121.609..."
874,Maurepas / Montigny-le-Bretonneux,FR,11210,51,"POLYGON ((3731181.439 2885511.912, 3731168.165..."
875,Maurepas / Montigny-le-Bretonneux,FR,11210,80,"POLYGON ((3731239.168 2885485.170, 3731222.682..."


In [14]:
ua['UA_area_m2'] = round(ua.geometry.area,2)
SA_ua = gpd.overlay(SA_mask,ua,how='intersection')  

In [15]:
SA_ua['final_area_m2'] = round(SA_ua.geometry.area,2)
SA_ua['area_prop'] = SA_ua['final_area_m2']/SA_ua['UA_area_m2'] # capture proportion from original geometry
SA_ua['Pop2018_poly'] = SA_ua['Pop2018']*SA_ua['area_prop']
round(SA_ua.Pop2018_poly.sum(),0)

54028.0

In [16]:
ua.head()

,code_2018,Pop2018,geometry,UA_area_m2
0,21000,0,"MULTIPOLYGON (((3730026.813 2884036.647, 37300...",4.767161e+04
1,21000,1,"MULTIPOLYGON (((3729660.852 2884957.545, 37296...",4.309398e+05
2,12220,0,"MULTIPOLYGON (((3690458.999 2887865.449, 36904...",3.443834e+08
3,11230,30,"MULTIPOLYGON (((3729140.131 2885631.625, 37291...",9.476450e+03
4,14200,4,"MULTIPOLYGON (((3728909.508 2886296.253, 37289...",4.494178e+05
...,...,...,...,...
1818,11210,663,"MULTIPOLYGON (((3736438.255 2886133.961, 37364...",6.625567e+04
1819,14200,38,"MULTIPOLYGON (((3736838.121 2885916.147, 37367...",9.074730e+04
1820,31000,0,"MULTIPOLYGON (((3733571.120 2888005.240, 37335...",4.822790e+06
1821,14200,17,"MULTIPOLYGON (((3740045.891 2886208.415, 37400...",5.639415e+05


In [17]:
SA_uc = gpd.clip(SA_mask,urban_center)  

In [18]:
SA_uc.head()

,geometry,HDENS_NAME,CNTR_CODE
HDENS_CLST,,,
GEOSTAT21_443,"MULTIPOLYGON (((3730778.938 2881010.228, 37307...",Maurepas / Montigny-le-Bretonneux,FR


In [20]:
SA_uc_ua = gpd.overlay(SA_uc,ua,how='intersection') 

In [21]:
SA_uc_ua

,HDENS_NAME,CNTR_CODE,code_2018,Pop2018,UA_area_m2,geometry
0,Maurepas / Montigny-le-Bretonneux,FR,21000,0,4.767161e+04,"POLYGON ((3730027.052 2884037.266, 3730026.813..."
1,Maurepas / Montigny-le-Bretonneux,FR,12220,0,3.443834e+08,"MULTIPOLYGON (((3739213.184 2881792.216, 37392..."
2,Maurepas / Montigny-le-Bretonneux,FR,11210,389,3.769582e+04,"POLYGON ((3729103.938 2882435.228, 3729076.019..."
3,Maurepas / Montigny-le-Bretonneux,FR,11210,160,2.575642e+04,"MULTIPOLYGON (((3728958.938 2882510.228, 37289..."
4,Maurepas / Montigny-le-Bretonneux,FR,12100,5,1.150870e+04,"POLYGON ((3729053.938 2882435.228, 3729063.938..."
...,...,...,...,...,...,...
872,Maurepas / Montigny-le-Bretonneux,FR,11210,54,7.287360e+03,"POLYGON ((3731000.000 2885630.552, 3731001.351..."
873,Maurepas / Montigny-le-Bretonneux,FR,11210,64,6.734230e+03,"POLYGON ((3731126.802 2885621.155, 3731121.609..."
874,Maurepas / Montigny-le-Bretonneux,FR,11210,51,8.317960e+03,"POLYGON ((3731181.439 2885511.912, 3731168.165..."
875,Maurepas / Montigny-le-Bretonneux,FR,11210,80,1.073271e+04,"POLYGON ((3731239.168 2885485.170, 3731222.682..."


In [25]:
SA_uc_ua.Pop2018.sum()

109945

In [22]:
SA_uc_ua['final_area_m2'] = round(SA_uc_ua.geometry.area,2)
SA_uc_ua['area_prop'] = SA_uc_ua['final_area_m2']/SA_uc_ua['UA_area_m2'] # capture proportion from original geometry
SA_uc_ua['Pop2018_poly'] = SA_uc_ua['Pop2018']*SA_uc_ua['area_prop']
round(SA_uc_ua.Pop2018_poly.sum(),0)

54028.0

In [23]:
SA_uc_ua

,HDENS_NAME,CNTR_CODE,code_2018,Pop2018,UA_area_m2,geometry,final_area_m2,area_prop,Pop2018_poly
0,Maurepas / Montigny-le-Bretonneux,FR,21000,0,4.767161e+04,"POLYGON ((3730027.052 2884037.266, 3730026.813...",0.05,0.000001,0.000000
1,Maurepas / Montigny-le-Bretonneux,FR,12220,0,3.443834e+08,"MULTIPOLYGON (((3739213.184 2881792.216, 37392...",1424863.95,0.004137,0.000000
2,Maurepas / Montigny-le-Bretonneux,FR,11210,389,3.769582e+04,"POLYGON ((3729103.938 2882435.228, 3729076.019...",302.51,0.008025,3.121736
3,Maurepas / Montigny-le-Bretonneux,FR,11210,160,2.575642e+04,"MULTIPOLYGON (((3728958.938 2882510.228, 37289...",3652.68,0.141816,22.690607
4,Maurepas / Montigny-le-Bretonneux,FR,12100,5,1.150870e+04,"POLYGON ((3729053.938 2882435.228, 3729063.938...",1408.49,0.122385,0.611924
...,...,...,...,...,...,...,...,...,...
872,Maurepas / Montigny-le-Bretonneux,FR,11210,54,7.287360e+03,"POLYGON ((3731000.000 2885630.552, 3731001.351...",7231.95,0.992396,53.589407
873,Maurepas / Montigny-le-Bretonneux,FR,11210,64,6.734230e+03,"POLYGON ((3731126.802 2885621.155, 3731121.609...",6734.23,1.000000,64.000000
874,Maurepas / Montigny-le-Bretonneux,FR,11210,51,8.317960e+03,"POLYGON ((3731181.439 2885511.912, 3731168.165...",8317.96,1.000000,51.000000
875,Maurepas / Montigny-le-Bretonneux,FR,11210,80,1.073271e+04,"POLYGON ((3731239.168 2885485.170, 3731222.682...",10732.71,1.000000,80.000000
